<a href="https://colab.research.google.com/github/minh-chaudang/IntroAI/blob/main/WaterSort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import itertools
import copy
import numpy as np
import heapq

# i commit

In [ ]:
class State:
  def __init__(self, bottles, parent = None, capacity = 0):
    self.bottles = bottles
    self.parent = parent
    if parent == None:
      self.capacity = max(len(bottle) for bottle in self.bottles)
      self.distance = 0
    else:
      self.capacity = parent.capacity
      self.distance = parent.distance + 1

  # Check if this is the goal
  def is_goal(self):
    return all(len(bottle) == 0 or len(bottle) == self.capacity and len(set(bottle)) == 1 for bottle in self.bottles)

  # Find all pourable bottle pairs
  def pourable_pairs(self):
    result = []
    for i in range(len(self.bottles)-1,-1,-1):
      for j in range(len(self.bottles)-1,-1,-1):
        if i != j:
          # One is containing and one is empty
          if len(self.bottles[i]) > 0 and len(self.bottles[j]) == 0: 
            result.append((i,j))
          # Two have the same tops
          if len(self.bottles[i]) > 0 and len(self.bottles[j]) > 0 and self.bottles[i][-1] == self.bottles[j][-1] and len(self.bottles[j]) < self.capacity:
            result.append((i,j))
    return result

  # Expand a state
  def expand(self):
    # If this state is expandible
    children = []
    for pair in self.pourable_pairs():
      child = State(copy.deepcopy(self.bottles), self)
      top = child.bottles[pair[0]][-1]
      while len(child.bottles[pair[0]]) > 0 and child.bottles[pair[0]][-1] == top and len(child.bottles[pair[1]]) < self.capacity:
        child.bottles[pair[0]].pop()
        child.bottles[pair[1]].append(top)
      children.append(child)
    return children

  # Estimated f = g + h
  def cost(self):
    h = 0
    for bottle in self.bottles:
      for i in range (len(bottle)-1):
        if bottle[i] != bottle[i+1]: h += 1
    return h + self.distance
  
  # Overide the comparator to push to a heap
  def __lt__(self, other):
        return self.cost() < other.cost()
   
  # Get path from root
  def getpath(self):
    path = [self]
    while path[-1].parent != None: path.append(path[-1].parent)
    return path

In [ ]:
level1 = State([['O'], ['O', 'O', 'O']])
level3 = State([['C', 'C', 'D', 'X'], ['X', 'C', 'D', 'X'], ['D','X','C','D'],[],[]])
level5 = State([['P','B','G','O'],['R','O','R','P'],['B','R','G','G'],['P','B','O','G'],['B','R','P','O'],[],[]])

In [ ]:
# Ordinary DFS
def DFShelper(stack, loop, visited):
  while len(stack) > 0 and stack[-1].bottles in visited: stack.pop()
  if len(stack) == 0: return -1, loop
  elif (stack[-1].is_goal()): 
    path = [stack[-1]]
    while path[-1].parent != None: path.append(path[-1].parent)
    path.reverse()
    return path, loop
  else: 
    this_state = stack[-1]
    stack.pop()
    visited.append(this_state.bottles)
    stack += this_state.expand()
    # print("Loop", loop, ":", this_state.bottles)
    return DFShelper(stack, loop+1, visited)

# For calling more conveniently
def DFS(initial_state):
  path, loop = DFShelper([initial_state], 0, [])
  if path == -1: print("No solution")
  else:
    print("DFS executed after", loop, "loops")
    for i in range(len(path)): print("Step", i, ":", path[i].bottles)

In [ ]:
def Astarhelper(heap, loop, visited):
  while len(heap) > 0 and heap[0].bottles in visited: heapq.heappop(heap)
  if len(heap) == 0: return -1, loop
  elif (heap[0].is_goal()):
    # print("Goal:", heap[0].bottles)
    path = [heap[0]]
    while path[-1].parent != None: path.append(path[-1].parent)
    path.reverse()
    return path, loop
  else:
    this_state = heap[0]
    heapq.heappop(heap)
    heap += this_state.expand()
    heapq.heapify(heap)
    visited.append(this_state.bottles)
    # print("Loop", loop, ":", this_state.bottles)
    return Astarhelper(heap, loop+1, visited)

def Astar(initial_state):
    path, loop = Astarhelper([initial_state], 0, [])
    if path == -1: print("No solution")
    else:
      print("DFS executed after", loop, "loops")
      for i in range(len(path)): print("Step", i, ":", path[i].bottles)
      print("A* executed after", loop, "loops")


In [ ]:
state = State([['C', 'C', 'D', 'X'], ['X', 'C', 'D', 'X'], ['D','X','C','D'],[],[]])
# path = Astar([state], 0, [])

Astar(state)



DFS executed after 692 loops
Step 0 : [['C', 'C', 'D', 'X'], ['X', 'C', 'D', 'X'], ['D', 'X', 'C', 'D'], [], []]
Step 1 : [['C', 'C', 'D', 'X'], ['X', 'C', 'D'], ['D', 'X', 'C', 'D'], ['X'], []]
Step 2 : [['C', 'C', 'D', 'X'], ['X', 'C', 'D', 'D'], ['D', 'X', 'C'], ['X'], []]
Step 3 : [['C', 'C', 'D'], ['X', 'C', 'D', 'D'], ['D', 'X', 'C'], ['X', 'X'], []]
Step 4 : [['C', 'C', 'D'], ['X', 'C', 'D', 'D'], ['D', 'X'], ['X', 'X'], ['C']]
Step 5 : [['C', 'C', 'D'], ['X', 'C', 'D', 'D'], ['D'], ['X', 'X', 'X'], ['C']]
Step 6 : [['C', 'C'], ['X', 'C', 'D', 'D'], ['D', 'D'], ['X', 'X', 'X'], ['C']]
Step 7 : [['C', 'C'], ['X', 'C'], ['D', 'D', 'D', 'D'], ['X', 'X', 'X'], ['C']]
Step 8 : [['C', 'C', 'C'], ['X'], ['D', 'D', 'D', 'D'], ['X', 'X', 'X'], ['C']]
Step 9 : [[], ['X'], ['D', 'D', 'D', 'D'], ['X', 'X', 'X'], ['C', 'C', 'C', 'C']]
Step 10 : [[], [], ['D', 'D', 'D', 'D'], ['X', 'X', 'X', 'X'], ['C', 'C', 'C', 'C']]
A* executed after 692 loops


In [ ]:
state = State(level5, None)
path = DFS([state], 0, [])

for i in range(len(path)):
  print("Step", i, ":", path[i].bottles)

DFS executed after 93 loops
Step 0 : [['P', 'B', 'G', 'O'], ['R', 'O', 'R', 'P'], ['B', 'R', 'G', 'G'], ['P', 'B', 'O', 'G'], ['B', 'R', 'P', 'O'], [], []]
Step 1 : [['P', 'B', 'G'], ['R', 'O', 'R', 'P'], ['B', 'R', 'G', 'G'], ['P', 'B', 'O', 'G'], ['B', 'R', 'P', 'O'], ['O'], []]
Step 2 : [['P', 'B'], ['R', 'O', 'R', 'P'], ['B', 'R', 'G', 'G'], ['P', 'B', 'O', 'G'], ['B', 'R', 'P', 'O'], ['O'], ['G']]
Step 3 : [['P', 'B'], ['R', 'O', 'R', 'P'], ['B', 'R'], ['P', 'B', 'O', 'G'], ['B', 'R', 'P', 'O'], ['O'], ['G', 'G', 'G']]
Step 4 : [['P', 'B'], ['R', 'O', 'R', 'P'], ['B', 'R'], ['P', 'B', 'O'], ['B', 'R', 'P', 'O'], ['O'], ['G', 'G', 'G', 'G']]
Step 5 : [['P', 'B'], ['R', 'O', 'R', 'P'], ['B', 'R'], ['P', 'B'], ['B', 'R', 'P', 'O'], ['O', 'O'], ['G', 'G', 'G', 'G']]
Step 6 : [['P'], ['R', 'O', 'R', 'P'], ['B', 'R'], ['P', 'B', 'B'], ['B', 'R', 'P', 'O'], ['O', 'O'], ['G', 'G', 'G', 'G']]
Step 7 : [['P', 'P'], ['R', 'O', 'R'], ['B', 'R'], ['P', 'B', 'B'], ['B', 'R', 'P', 'O'], ['O', 'O

In [ ]:
input1 = State([['X','C','X','C'],['C','X','C','X'],[]], None)

path = Astar([input1], 0, [])

for i in range(len(path)):
  print("Step", i, ":", path[i].bottles)


A* executed after 13 loops
Step 0 : [['X', 'C', 'X', 'C'], ['C', 'X', 'C', 'X'], []]
Step 1 : [['X', 'C', 'X', 'C'], ['C', 'X', 'C'], ['X']]
Step 2 : [['X', 'C', 'X'], ['C', 'X', 'C', 'C'], ['X']]
Step 3 : [['X', 'C'], ['C', 'X', 'C', 'C'], ['X', 'X']]
Step 4 : [['X', 'C', 'C', 'C'], ['C', 'X'], ['X', 'X']]
Step 5 : [['X', 'C', 'C', 'C'], ['C'], ['X', 'X', 'X']]
Step 6 : [['X'], ['C', 'C', 'C', 'C'], ['X', 'X', 'X']]
Step 7 : [['X', 'X', 'X', 'X'], ['C', 'C', 'C', 'C'], []]
